In [2]:
import pandas as pd
import nltk
import string
import os
from collections import Counter
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy import stats
from sklearn.preprocessing import OneHotEncoder

## Leitura dos dados

In [3]:
train_df = pd.read_csv('train.csv')
train_df.shape

(20800, 5)

In [4]:
test_df = pd.read_csv('test.csv')
test_df.shape

(5200, 4)

In [5]:
train_df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Obtendo vetor de documentos

In [6]:
porter_stemmer = PorterStemmer()

def postag_sentence(sentence_tokens):
    return nltk.pos_tag(sentence_tokens)

def postag_filter(sentence_tokens):
    pos_keep = ['NN', 'NNS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 
                'VBZ', 'NNP', 'NNPS','RB', 'RBR', 'RBS', 'JJ', 'JJR', 'JJS']
    sentence_tokens_filtered = [pt[0] for pt in postag_sentence(sentence_tokens) if pt[1] in pos_keep]
    return sentence_tokens_filtered

def stem_token(token):
    return porter_stemmer.stem(token)

def tokenize_doc(doc, use_pt_filter=False,
                      use_lowercase_filter=False,
                      use_stopwords_filter=False,
                      use_stemming_filter=False,
                      add_bigrams=False):
    sentences = sent_tokenize(doc)
    tokens = []
    for s in sentences:
        tk_sentence = word_tokenize(s)
        if use_pt_filter:
            tk_sentence = postag_filter(tk_sentence)
        if use_lowercase_filter:
            tk_sentence = [tk.lower() for tk in tk_sentence]
        if use_stopwords_filter:
            tk_sentence = [tk for tk in tk_sentence if tk not in stopwords.words('english')]
        if use_stemming_filter:
            tk_sentence = [stem_token(tk) for tk in tk_sentence ]
        tokens.extend(tk_sentence)
        if add_bigrams:
            tokens.extend(nltk.bigrams(tk_sentence))
        
    return tokens
        
def remove_punctuation(tokens):
    return [ t for t in tokens if t not in string.punctuation ]

def get_vocabulary_tokenized_from_docs(tk_docs):
    vocabulary = Counter()
    for tk_d in tk_docs:
        for tk in tk_d:
            vocabulary[tk] += 1
    return vocabulary

In [7]:
docs = []
authors = []
authors_list = list(train_df['author'].unique())
for index, row in train_df.iterrows():
    docs.append(' '.join([str(row['title']), str(row['text'])]).replace("’", "'"))
    authors.append(authors_list.index(row['author']))

In [8]:
authors_matrix = np.zeros((len(authors), len(authors_list)))
for i in range(len(authors)):
    author_onehotencoded = np.zeros(len(authors_list))
    author_onehotencoded[authors[i]] = 1
    authors_matrix[i,:] = author_onehotencoded

## Vocabulario e reducao de dimensionalidade por vocabulario

#### Sem Pos-Tagging, Sem Lowercase, Sem remoção Stopwords, Sem Stemming

In [ ]:
tk_docs = [tokenize_doc(d, False, False, False, False) for d in docs]
vocabulary = get_vocabulary_tokenized_from_docs(tk_docs)
print('Tamanho do vocabulario inicial:', len(vocabulary.keys()))

#### Com Pos-Tagging, Sem Lowercase, Sem remoção Stopwords, Sem Stemming

In [ ]:
tk_docs = [tokenize_doc(d, True, False, False, False) for d in docs]
vocabulary = get_vocabulary_tokenized_from_docs(tk_docs)
print('Tamanho do vocabulario inicial:', len(vocabulary.keys()))

#### Com Pos-Tagging, Com Lowercase, Sem remoção Stopwords, Sem Stemming

In [ ]:
tk_docs = [tokenize_doc(d, True, True, False, False) for d in docs]
vocabulary = get_vocabulary_tokenized_from_docs(tk_docs)
print('Tamanho do vocabulario com lowercase:', len(vocabulary.keys()))

#### Com Pos-Tagging, Com Lowercase, Com remoção Stopwords, Sem Stemming

In [ ]:
tk_docs = [tokenize_doc(d, True, True, True, False) for d in docs]
vocabulary = get_vocabulary_tokenized_from_docs(tk_docs)
print('Tamanho do vocabulario com lowercase:', len(vocabulary.keys()))

#### Com Pos-Tagging, Com Lowercase, Com remoção Stopwords, Com Stemming

In [ ]:
tk_docs = [tokenize_doc(d, True, True, True, True) for d in docs]
vocabulary = get_vocabulary_tokenized_from_docs(tk_docs)
print('Tamanho do vocabulario com lowercase:', len(vocabulary.keys()))

#### Com Pos-Tagging, Com Lowercase, Com remoção Stopwords, Com Stemming e Bigramas

In [9]:
tk_docs = [tokenize_doc(d, True, True, True, True, add_bigrams=True) for d in docs]
vocabulary = get_vocabulary_tokenized_from_docs(tk_docs)
print('Tamanho do vocabulario com lowercase:', len(vocabulary.keys()))

Tamanho do vocabulario com lowercase: 3545517


### Filtrando por frequencia

In [11]:
labels, values = zip(*vocabulary.items())
indexes = np.arange(len(labels))
values_list = np.array(values)

In [12]:
mu, std = stats.norm.fit(values_list)
max_cut = np.ceil(stats.norm.ppf(0.9, mu, std))
min_cut = 10

In [13]:
keys = list(vocabulary.keys())
print('Vocab Size before cut:', len(keys))
for k in keys:
    if vocabulary[k] > max_cut:
        vocabulary.pop(k, None)
        continue
    if vocabulary[k] < min_cut:
        vocabulary.pop(k, None)
        
keys = list(vocabulary.keys())
print('Vocab Size after cut:', len(keys))

Vocab Size before cut: 3545517
Vocab Size after cut: 124683


## Matriz de count do texto

In [ ]:
text_matrix = np.zeros((len(docs), len(list(vocabulary.keys()))))
vocab_size = len(keys)
n_docs = len(tk_docs)
for i in range(n_docs):
    row_vec = np.zeros(vocab_size)
    for tk in tk_docs[i]:
        try:
            token_index = keys.index(tk)
            row_vec[token_index] += 1
        except Exception as ex:
            continue
    
    print('Processed row:', (i+1), '/', n_docs)
    text_matrix[i, :] = row_vec

In [55]:
np.save('kaggle_countmatrix', text_matrix)

## Checkpoint - matriz de texto salva

In [ ]:
text_matrix = np.load('kaggle_countmatrix.npy')

In [10]:
text_matrix.shape

NameError: name 'text_matrix' is not defined

### Juntando matriz de texto com de autores

In [ ]:
authors_matrix.shape

In [ ]:
reduc_dim = 200
from sklearn.decomposition import PCA

